In [1]:
import sys 
from pathlib import Path
src_dir = Path.cwd().parent
sys.path.append(str(src_dir))

from models.LP import LabelPropagation , LabelSpreading
from utilits.utilits import *
from models.Classifier_model import ClassifierModel
from models.simCSE import SimCSE

## load Datasets

In [2]:
olid_dataset = prepare_data("../dataset/")
olid_dataset

{'train': Dataset({
     features: ['tweets', 'labels', 'labels_name'],
     num_rows: 13240
 }),
 'test': Dataset({
     features: ['tweets', 'labels', 'labels_name'],
     num_rows: 3887
 })}

In [3]:
#split data to train_dev 
training_data_m1 = split_data(olid_dataset["train"] , annotated_data_prec=0.8)
training_data_m1

DatasetDict({
    train: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 10592
    })
    test: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 2648
    })
})

### Train Classifier Model on all oiled_Dataset 

In [4]:
m1 = ClassifierModel()
m1.train(training_data_m1["train"],
         training_data_m1["test"] ,
         num_epochs=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.492200,0.410793,0.824396,0.801083,0.650549,0.718011
2,0.394200,0.410939,0.825906,0.786718,0.676923,0.727702


In [5]:
m1_metrics = m1.test(olid_dataset["test"])
m1_metrics

  0%|          | 0/1 [00:00<?, ?ba/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: labels_name, tweets. If labels_name, tweets are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3887
  Batch size = 8


{'test_loss': 0.19897423684597015,
 'test_accuracy': 0.9269359403138667,
 'test_precision': 0.8015151515151515,
 'test_recall': 0.9796296296296296,
 'test_f1': 0.8816666666666666,
 'test_runtime': 4.3034,
 'test_samples_per_second': 903.231,
 'test_steps_per_second': 112.933}

In [6]:
all_m_metrics = dict()
all_m_metrics["All_dataset"] = m1_metrics

## annotate ulabeled data with self-supervised learning with LP 

In [4]:
# 10% of data labeld
annoteted_data = split_data(olid_dataset["train"] ,annotated_data_prec = 0.1)
annoteted_data

DatasetDict({
    train: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 1324
    })
    test: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 11916
    })
})

In [5]:
from models.Label_Data import DataLabeling

In [6]:
LD = DataLabeling()
preds_labels =LD.generate_labels(annoteted_data["train"]["tweets"],
                                 annoteted_data["train"]["labels"],
                                 annoteted_data["test"]["tweets"])

100%|██████████| 187/187 [00:28<00:00,  6.61it/s]


In [7]:
training_annoteted_data = create_dataset_of_label_propagation(annoteted_data["test"]["tweets"] ,
                                                             preds_labels)
training_annoteted_data = split_data(training_annoteted_data , annotated_data_prec = 0.8)
training_annoteted_data

DatasetDict({
    train: Dataset({
        features: ['tweets', 'labels'],
        num_rows: 9532
    })
    test: Dataset({
        features: ['tweets', 'labels'],
        num_rows: 2384
    })
})

In [34]:
m3 = ClassifierModel()
m3.train(training_annoteted_data["train"],
         training_annoteted_data["test"] ,
         num_epochs=3)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.3",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.485300,0.423091,0.792366,0.632737,0.668605,0.650177
2,0.347100,0.401515,0.815856,0.710660,0.610465,0.656763
3,0.276100,0.412167,0.816695,0.687034,0.670058,0.678440


In [35]:
m2_metrics = m3.test(olid_dataset["test"])

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweets, labels_name. If tweets, labels_name are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3887
  Batch size = 8


In [36]:
all_m_metrics["10%_labeled_data"] = m2_metrics
all_m_metrics

{'All_dataset': {'test_loss': 0.20425701141357422,
  'test_accuracy': 0.9243632621559043,
  'test_precision': 0.7946026986506747,
  'test_recall': 0.9814814814814815,
  'test_f1': 0.8782104391052195,
  'test_runtime': 10.9008,
  'test_samples_per_second': 356.58,
  'test_steps_per_second': 44.584},
 '10%_labeled_data': {'test_loss': 0.3574120104312897,
  'test_accuracy': 0.8525855415487522,
  'test_precision': 0.7463556851311953,
  'test_recall': 0.7111111111111111,
  'test_f1': 0.728307254623044,
  'test_runtime': 10.5885,
  'test_samples_per_second': 367.098,
  'test_steps_per_second': 45.899}}

# with 20% annoteted

In [37]:
annoteted_data = split_data(olid_dataset["train"] , annotated_data_prec= 0.2)
annoteted_data

DatasetDict({
    train: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 2648
    })
    test: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 10592
    })
})

In [38]:
LD = DataLabeling()
preds_labels =LD.generate_labels(annoteted_data["train"]["tweets"],
                                 annoteted_data["train"]["labels"],
                                 annoteted_data["test"]["tweets"])

loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1480925a23f7db13cea1c830922dbd4173c2a1ccab8c57cbb36a1ea693164879.01dc297b74ef2153586ff6f1113a3309f339a11f1cef9d887ae2314924e8d17e
Model config RobertaConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-roberta-base",
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "v

In [39]:
training_annoteted_data = create_dataset_of_label_propagation(annoteted_data["test"]["tweets"] ,
                                                             preds_labels)
training_annoteted_data = split_data(training_annoteted_data , annotated_data_prec = 0.8)
training_annoteted_data

DatasetDict({
    train: Dataset({
        features: ['tweets', 'labels'],
        num_rows: 8473
    })
    test: Dataset({
        features: ['tweets', 'labels'],
        num_rows: 2119
    })
})

In [40]:
m4 = ClassifierModel()
m4.train(training_annoteted_data["train"],
         training_annoteted_data["test"] ,
         num_epochs=3)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.3",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.470600,0.404897,0.810288,0.785498,0.439932,0.563991
2,0.338400,0.368000,0.834828,0.735812,0.636210,0.682396
3,0.276300,0.380412,0.837659,0.717047,0.690355,0.703448


In [41]:
m3_metrics = m4.test(olid_dataset["test"])
all_m_metrics["20%_labeled_data"] = m3_metrics

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweets, labels_name. If tweets, labels_name are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3887
  Batch size = 8


In [43]:
import pandas as pd 

In [50]:
pd.DataFrame(all_m_metrics).iloc[1:5]

,All_dataset,10%_labeled_data,20%_labeled_data
test_accuracy,0.924363,0.852586,0.858760
test_precision,0.794603,0.746356,0.742909
test_recall,0.981481,0.711111,0.751852
test_f1,0.878210,0.728307,0.747354
